# Spotify Song Recommendation Algorithm

In [1]:
import os
import json
#!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

## Spotify ID

In [2]:
cid ='d8762235995b4ddcbc0df90739df9a63' # Client ID; copy this from your app created on beta.developer.spotify.com
secret = '887a4eccdbde4195bfc0f7f4c472fe70' # Client Secret; copy this from your app
username = '12155311349' # Your Spotify username

#for avaliable scopes see https://developer.spotify.com/web-api/using-scopes/
scope = 'user-library-read playlist-modify-public playlist-read-private'
redirect_uri='https://localhost:8080/callback/'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

token = util.prompt_for_user_token(username,scope,client_id=cid,client_secret=secret,redirect_uri="https://localhost:8080/callback/")
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

## Request Song From User

In [ ]:
song_name = input("Search a song for recommendations: ")

### Getting Relevant Data For Dataset

In [ ]:
# Define the search query
search_results = sp.search(q=song_name, type='track', limit=1)
track_id = search_results['tracks']['items'][0]['id']

# Get the audio features, year, and popularity of the song
track_id = search_results['tracks']['items'][0]['id']
audio_features = sp.audio_features([track_id])[0]
year = search_results['tracks']['items'][0]['album']['release_date'][:4]
popularity = search_results['tracks']['items'][0]['popularity']

# Find similar songs based on audio features, year, and popularity
similar_songs = sp.recommendations(seed_tracks=[track_id], target_popularity=popularity, limit=100, **audio_features)
# similar_songs = sp.recommendations(seed_tracks=[track_id], limit=100, **audio_features)

# Extract relevant info from recommendations and store in DataFrame
columns = ['song name', 'artist', 'popularity', 'year', 'iD', 'danceability', 'energy','loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
data = []
for i, track in enumerate(similar_songs['tracks']):
    name = track['name']
    artist = track['artists'][0]['name']
    popularity = track['popularity']
    year = track['album']['release_date'][:4]
    track_id = track['id']
    audio_features = sp.audio_features([track_id])[0]
    danceability = audio_features['danceability']
    loudness = audio_features['loudness']
    energy = audio_features['energy']
    speechiness = audio_features['speechiness']
    acousticness = audio_features['acousticness']
    instrumentalness = audio_features['instrumentalness']
    liveness = audio_features['liveness']
    valence = audio_features['valence']
    tempo = audio_features['tempo']
    data.append([name, artist, popularity, year, track_id, danceability, loudness,energy, speechiness, acousticness, instrumentalness, liveness, valence, tempo])
df = pd.DataFrame(data, columns=columns)

# Scale between 0 and 1
df["tempo"] = (df["tempo"] / df["tempo"].max())
df["loudness"] = (df["loudness"] / df["loudness"].max())
df["energy"] = (df["energy"] / df["energy"].min())
df["popularity"] = (df["popularity"] / 100)

In [ ]:
df

## Using Nearest Neighbors For Best Recommendation

In [ ]:
from sklearn.neighbors import NearestNeighbors

# Define the features to use for the model
features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

# Create a new dataframe with only the features to use for the model
df_model = df[features]

# Fit the k-nearest neighbors model using the new dataframe
knn = NearestNeighbors(n_neighbors=5)
knn.fit(df_model)

# Find the k nearest neighbors for the user-requested song
song_data = df[df['iD'] == track_id][features]
distances, indices = knn.kneighbors(song_data)

# Create a new dataframe with the nearest neighbors and their features
neighbor_data = df.iloc[indices[0]]

neighbor_data


import seaborn as sns

# Compute the correlation matrix
corr_matrix = df.drop(['song name', 'artist', 'year', 'iD'], axis=1).corr()

# Visualize the correlation matrix using a heatmap
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')

import seaborn as sns

# Compute the correlation matrix
corr_matrix = neighbor_data.drop(['song name', 'artist', 'year', 'iD'], axis=1).corr()

# Visualize the correlation matrix using a heatmap
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')

## Plotting the Final Solution

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Get audio features of original song
song_data = df.iloc[0][['popularity','danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]
song_features = np.array(song_data.tolist())

# Get audio features of neighbors
neighbor_data = neighbor_data.iloc[0:10][['popularity','danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]
neighbor_features = np.array(neighbor_data.values)

# Define feature labels
labels = ['Popularity','Danceability', 'Energy', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo']

# Define angles and labels for radar chart
angles = np.linspace(0, 2*np.pi, len(song_features)+1, endpoint=True)

# Create figure and subplot for radar chart
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, polar=True)

# Plot radar chart for song
ax.plot(angles[:], np.append(song_features, song_features[0]), 'o-', linewidth=2, label='Original Song')
ax.fill(angles, np.append(song_features, song_features[0]), alpha=0.25)

# Plot radar chart for neighbors in red
ax.plot(angles[:], np.append(neighbor_features.mean(axis=0), neighbor_features.mean(axis=0)[0]), 'o-', linewidth=2, color='red', label='Neighbor Songs')
ax.fill(angles, np.append(neighbor_features.mean(axis=0), neighbor_features.mean(axis=0)[0]), alpha=0.25, color='red')

# Set tick locations and labels
ax.set_xticks(angles[:-1])
ax.set_xticklabels(labels)

# Set title and legend
ax.set_title('Audio Features Comparison', fontsize=16, fontweight='bold')
ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
plt.show()
